### NVlink 유무체크 방법 
- NV#이 포함되어있어야하는데 포함되지 않음.
- $ nvidia-smi topo -m

## Dask cpu 설정 & Dask-cuda GPU

- (CPU)jupyterlab에서 클러스터를 추가한후 port설정을 셋팅한후 실행한다.
- (GPU)
   : 터미널을 연 후에, 가상환경을 설정한다.
   $ conda activate dask_cuda
   $ dask-scheduler 
   !/data01/anaconda3/envs/dask_gpu/bin/dask-scheduler

## NVLink있을시 필수적으로 ucx프로토콜 사용, 그러나 nvlink없을시에도 tcp프로톸ㄹ보다 ucx프로토콜이 성능이 더 빠르다고 한다.
- https://www.dask.org/blog/experiments-in-high-performance-networking-with-ucx-and-dgx
- $ dask-scheduler --protocol ucx

- $ dask-worker ucx://127.0.0.1:8786

In [4]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:32855")
client

SyntaxError: EOL while scanning string literal (3528771002.py, line 3)

In [ ]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Clientgigig

In [ ]:
cluster = LocalCUDACluster()

In [ ]:
client = Client(cluster)

In [ ]:
import sys

In [8]:
print(sys.executable)

/data01/anaconda3/envs/dask_cuda_xgboost/bin/python


In [9]:
!/data01/anaconda3/envs/dask_gpu/bin/python --version

Python 3.9.12


In [10]:
print(sys.path)

['/home/gigascake/dask_Example', '/data01/anaconda3/envs/dask_cuda_xgboost/lib/python39.zip', '/data01/anaconda3/envs/dask_cuda_xgboost/lib/python3.9', '/data01/anaconda3/envs/dask_cuda_xgboost/lib/python3.9/lib-dynload', '', '/home/gigascake/.local/lib/python3.9/site-packages', '/data01/anaconda3/envs/dask_cuda_xgboost/lib/python3.9/site-packages']


In [11]:
sys.path.append("/data01/anaconda3/envs/dask_gpu/bin")

In [12]:
sys.path

['/home/gigascake/dask_Example',
 '/data01/anaconda3/envs/dask_cuda_xgboost/lib/python39.zip',
 '/data01/anaconda3/envs/dask_cuda_xgboost/lib/python3.9',
 '/data01/anaconda3/envs/dask_cuda_xgboost/lib/python3.9/lib-dynload',
 '',
 '/home/gigascake/.local/lib/python3.9/site-packages',
 '/data01/anaconda3/envs/dask_cuda_xgboost/lib/python3.9/site-packages',
 '/data01/anaconda3/envs/dask_gpu/bin']

## Dask-cuda GPU

- 터미널을 연 후에, 가상환경을 설정한다.
- $ dask-scheduler를 실행한다.
- !/data01/anaconda3/envs/dask_gpu/bin/dask-scheduler

In [7]:
# dask-scheduler에서 Open된 port를 소스코드에 설정한 후에 실핸한다.
from dask.distributed import Client

client = Client("tcp://172.17.0.2:8786")
client

Connection method: Direct,
Dashboard: http://172.17.0.2:8787/status,
Comm: tcp://172.17.0.2:8786,Workers: 0
Dashboard: http://172.17.0.2:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [8]:
import xgboost.dask

In [9]:
from dask_cuda import LocalCUDACluster
import dask_cudf
from dask.distributed import Client
from dask import array as da
from dask import dataframe as dd
import xgboost as xgb
from xgboost import dask as dxgb
from xgboost.dask import DaskDMatrix


def using_dask_matrix(client: Client, X, y):
    # DaskDMatrix acts like normal DMatrix, works as a proxy for local
    # DMatrix scatter around workers.
    dtrain = DaskDMatrix(client, X, y)

    # Use train method from xgboost.dask instead of xgboost.  This
    # distributed version of train returns a dictionary containing the
    # resulting booster and evaluation history obtained from
    # evaluation metrics.
    output = xgb.dask.train(client,
                            {'verbosity': 2,
                             # Golden line for GPU training
                             'tree_method': 'gpu_hist'},
                            dtrain,
                            num_boost_round=8, evals=[(dtrain, 'train')])
    bst = output['booster']
    history = output['history']

    # you can pass output directly into `predict` too.
    prediction = xgb.dask.predict(client, bst, dtrain)
    print('Evaluation history:', history)
    return prediction


def using_quantile_device_dmatrix(client: Client, X, y):
    '''`DaskDeviceQuantileDMatrix` is a data type specialized for `gpu_hist`, tree
     method that reduces memory overhead.  When training on GPU pipeline, it's
     preferred over `DaskDMatrix`.

    .. versionadded:: 1.2.0

    '''
    # Input must be on GPU for `DaskDeviceQuantileDMatrix`.
    X = dask_cudf.from_dask_dataframe(dd.from_dask_array(X))
    y = dask_cudf.from_dask_dataframe(dd.from_dask_array(y))

    # `DaskDeviceQuantileDMatrix` is used instead of `DaskDMatrix`, be careful
    # that it can not be used for anything else other than training.
    dtrain = dxgb.DaskDeviceQuantileDMatrix(client, X, y)
    output = xgb.dask.train(client,
                            {'verbosity': 2,
                             'tree_method': 'gpu_hist'},
                            dtrain,
                            num_boost_round=4)

    prediction = xgb.dask.predict(client, output, X)
    return prediction




### API설정정보는 다음을 참조한다.
- https://docs.rapids.ai/api/dask-cuda/stable/api.html

### NVLINK가 없는 서버의 GPU용

In [10]:
if __name__ == '__main__':
    # `LocalCUDACluster` is used for assigning GPU to XGBoost processes.  Here
    # `n_workers` represents the number of GPUs since we use one GPU per worker
    # process.
    with LocalCUDACluster(n_workers=4, threads_per_worker=8) as cluster:
        with Client(cluster) as client:
            # generate some random data for demonstration
            m = 100000
            n = 100
            X = da.random.random(size=(m, n), chunks=10000)
            y = da.random.random(size=(m, ), chunks=10000)

            print('Using DaskQuantileDMatrix')
            from_ddqdm = using_quantile_device_dmatrix(client, X, y)
            print('Using DMatrix')
            from_dmatrix = using_dask_matrix(client, X, y)

/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44981 instead
  warnings.warn(
2022-10-11 07:34:27,722 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-10-11 07:34:27,722 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-10-11 07:34:27,723 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-10-11 07:34:27,723 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-10-11 07:34:27,755 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-10-11 07:34:27,755 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-10-11 07:34:27,776 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-10-11 07:34:27,776 - distributed.preloading - INFO - Import preloa

Using DaskQuantileDMatrix


/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[07:34:29] task [xgboost.dask]:tcp://127.0.0.1:33899 got new rank 0
[07:34:29] task [xgboost.dask]:tcp://127.0.0.1:43745 got new rank 1
[07:34:29] task [xgboost.das

Using DMatrix


/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[07:34:30] task [xgboost.dask]:tcp://127.0.0.1:43745 got new rank 0
[07:34:30] task [xgboost.dask]:tcp://127.0.0.1:41979 got new rank 1
[07:34:30] task [xgboost.das

[0]	train-rmse:0.28793
[1]	train-rmse:0.28744
[2]	train-rmse:0.28708
[3]	train-rmse:0.28662
[4]	train-rmse:0.28608
[5]	train-rmse:0.28559
[6]	train-rmse:0.28510
[7]	train-rmse:0.28474
Evaluation history: {'train': OrderedDict([('rmse', [0.28792558979928484, 0.28744126946665394, 0.28708283483970687, 0.28661894821098116, 0.28608470731674607, 0.28559380056328953, 0.28509500199394683, 0.28474362712465295])])}


2022-10-11 07:34:31,351 - distributed.worker - WARNING - Heartbeat to scheduler failed
Traceback (most recent call last):
  File "/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/comm/tcp.py", line 223, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/worker.py", line 1159, in heartbeat
    response = await retry_operation(
  File "/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/utils_comm.py", line 383, in retry_operation
    return await retry(
  File "/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/utils_comm.py", line 368, in retry
    return await coro()
  File "/data01/anaconda3/envs/dask_cuda/lib/python3.9/site-packages/distributed/core.py", line 1153, in se

### NVLINK가 있는 서버의 GPU용

In [ ]:
if __name__ == '__main__':
    # `LocalCUDACluster` is used for assigning GPU to XGBoost processes.  Here
    # `n_workers` represents the number of GPUs since we use one GPU per worker
    # process.
    with LocalCUDACluster(n_workers=4, threads_per_worker=8, rmm_pool_size="1GB", protocol="ucx", enable_tcp_over_ucx=True, enable_nvlink=True) as cluster:
        with Client(cluster) as client:
            # generate some random data for demonstration
            m = 100000
            n = 100
            X = da.random.random(size=(m, n), chunks=10000)
            y = da.random.random(size=(m, ), chunks=10000)

            print('Using DaskQuantileDMatrix')
            from_ddqdm = using_quantile_device_dmatrix(client, X, y)
            print('Using DMatrix')
            from_dmatrix = using_dask_matrix(client, X, y)